In [ ]:
from datascience import *
%matplotlib inline
path_data = '../../../assets/data/'
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=np.VisibleDeprecationWarning)

# Lecture 21

## Discussion Question: Super Soda

Recall the problem setup: 

* We observe 91/200 taste testers prefer Super Soda
* We expected it to be 50/50 — is the observed result due to chance or do they truly not like Super Soda as much?
    * Null: 50% of the population prefers Super Soda and 50% do not.
    * Alternative: Less than 50% of the population prefers Super Soda.
* Test statistic: Number of people out of 200 who prefer Super Soda
* P-value: ?

In [ ]:
# Writing a function to simulate under the null hypothesis (50-50 split of Super Soda fans)

def simulate_one_count(sample_size):
    return np.count_nonzero(np.random.choice(['H', 'T'], sample_size) == 'H')

simulate_one_count(200)

In [ ]:
# Generate distribution under the null hypothesis

num_simulations = 10000
counts = make_array()

for i in np.arange(num_simulations):
    counts = np.append(counts, simulate_one_count(200))

In [ ]:
# What does the distribution under the null look like?

trials = Table().with_column('Number of Heads', counts)
trials.hist(right_end=91)
plots.ylim(-0.001, 0.055)
plots.scatter(91, 0, color='red', s=40, zorder=3)
plots.title('Prediction Under the Null');

In [ ]:
# Calculate the p-value — which values support the alternative hypothesis?

np.count_nonzero(counts <= 91)/len(counts)

Make a conclusion:

# Things that can affect the result of a hypothesis test

## Changing the number of simulations

In [ ]:
# Keeping the data fixed, we can re-run the test with a new simulation under the null

def run_test(num_simulations, sample_size):
    counts = make_array()
    for i in np.arange(num_simulations):
        counts = np.append(counts, simulate_one_count(sample_size))
    return counts

counts = run_test(10000, 200)
np.count_nonzero(counts <= 91)/len(counts)

In [ ]:
# Trying out 3 numbers of simulations: 100, 1000, 10000

# We'll conduct 20 hypothesis tests at each number of simulations.

tests = Table(['simulations', 'p-value for 91'])
for num_sims in [100, 1000, 10000]:
    for k in np.arange(20):
        counts = run_test(num_sims, 200)
        tests = tests.with_row([
            num_sims, 
            np.count_nonzero(counts <= 91)/len(counts),
        ])
tests.group('simulations', np.mean)

In [ ]:
# For larger numbers of simulations, p-values are more consistent

tests.hist(1, group='simulations')

In [ ]:
# Since a large number of simulations provides a good estimate of the
# theoretical distribution of the test statistic under the null hypothesis
num_sims = 10000
counts_1 = run_test(num_sims, 200)
counts_2 = run_test(num_sims, 200)
t = Table().with_columns(
    'Experiment', [1] * num_sims + [2] * num_sims,
    'Number of Heads', np.append(counts_1, counts_2))
t.hist(1, group='Experiment', bins=np.arange(70.5, 131, 1))

## Changing the sample size

This is different from changing the number of simulations!

Let's say we're now running a taste test of 200\*2 = 400 people. We observe that 91\*2 = 182 of them prefer Super Soda.

In [ ]:
num_simulations = 10000
counts = make_array()

for i in np.arange(num_simulations):
    counts = np.append(counts, simulate_one_count(400))
    
# What does the distribution under the null look like?

trials = Table().with_column('Number of Heads', counts)
trials.hist(right_end=182)
plots.ylim(-0.001, 0.055)
plots.scatter(182, 0, color='red', s=40, zorder=3)
plots.title('Prediction Under the Null');

In [ ]:
np.count_nonzero(counts <= 182)/len(counts)

The p-value decreased by quite a lot! Think about why this is the case. An example: It's more unusual to get 600 heads out of 1000 coin tosses than 6 heads out of 10 coin tosses.

## Changing the truth about the population

We'll use simulation to explore what happens when our population changes. Since we know the truth about the population, we can use this to figure out how accurate our results are.

In [ ]:
# Suppose that the true proportion of people who prefer Super Soda is 45%
true_proportion = 0.45
true_distribution = make_array(true_proportion, 1 - true_proportion)
true_distribution

In [ ]:
# Taste tests with 200 people will give varioius numbers of people who prefer Super Soda
sample_size = 200
sample_proportions(sample_size, true_distribution) * sample_size

In [ ]:
# If you run a taste test for 200 people, what might you conclude?
def run_experiment(num_simulations, sample_size, true_proportion):
    # Collect data
    true_distribution = make_array(true_proportion, 1 - true_proportion)
    taste_test_results = sample_proportions(sample_size, true_distribution) * sample_size
    observed_stat_from_this_sample = taste_test_results.item(0)
    
    # Conduct hypothesis test
    counts = run_test(num_simulations, sample_size)
    p_value = np.count_nonzero(counts <= observed_stat_from_this_sample) / len(counts)
    return p_value

run_experiment(10000, 200, 0.45)

In [ ]:
# Let's imagine running our taste test over and over again to see how often we reject the null
true_proportion = 0.45
sample_size = 200
p_values = make_array()
for k in np.arange(100):
    p_value = run_experiment(10000, sample_size, true_proportion)
    p_values = np.append(p_values, p_value)
Table().with_column('P-value', p_values).hist(0, bins=20)

# proportion of experiments where we rejected the null
np.mean(p_values <= 0.05)

In [ ]:
# Let's imagine running our taste test over and over again to see how often we reject the null
true_proportion = 0.48
sample_size = 200
p_values = make_array()
for k in np.arange(100):
    p_value = run_experiment(10000, sample_size, true_proportion)
    p_values = np.append(p_values, p_value)
Table().with_column('P-value', p_values).hist(0, bins=20)

# proportion of experiments where we rejected the null
np.mean(p_values <= 0.05)

In [ ]:
# Let's imagine running our taste test over and over again to see how often we reject the null
true_proportion = 0.3
sample_size = 200
p_values = make_array()
for k in np.arange(100):
    p_value = run_experiment(10000, sample_size, true_proportion)
    p_values = np.append(p_values, p_value)
Table().with_column('P-value', p_values).hist(0, bins=20)

# proportion of experiments where we rejected the null
np.mean(p_values <= 0.05)